In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery


In [3]:
# Construct a BigQuery client object.
## Dá erro de não tiver o CLI instalado. 
client = bigquery.Client()

In [ ]:
#para excluir tabelas
#client.delete_table('perfil_remuneracao.ESTADIC_policial_civil_habitantes', not_found_ok=True)

## Instruções Gerais uso do GBQ

#### Criando um dataset

Abaixo se cria o identificador do dataset com o nome do repositório antes.
O `format` substitui o que está entre {} pelo que está como parâmetro. 

In [63]:
dataset_id = "{}.conjunto_teste2".format(client.project)

In [65]:
# Transforma em google.cloud.bigquery.dataset.Dataset 
dataset = bigquery.Dataset(dataset_id)
# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset repositoriodedadosgpsp.conjunto_teste2


#### Exemplo de como acessar tabelas do projeto para tratamento

In [4]:
# A função abaixo precisa do conjunto de dados e do nome da tabela a ser requisitada. 

table = client.get_table('repositoriodedadosgpsp.perfil_remuneracao.RAIS_vinculos_publicos_grafico') 

In [5]:
## Lendo table com query

df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Table(TableReference(DatasetReference('repositoriodedadosgpsp', 'perfil_remuneracao'), 'RAIS_vinculos_publicos_grafico'))

In [6]:
## Transforma tabela bq em dataframe
df = client.list_rows(table).to_dataframe()

In [8]:
df.columns

Index(['ano', 'sexo', 'poderes', 'grau_instrucao', 'esfera', 'faixa_etaria',
       'sigla_uf', 'carga_horaria', 'quantidade_vinculos'],
      dtype='object')

#### Chamar um df externo para tratamento

In [33]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')

In [35]:
df = pd.read_csv('IPEA_proporcao_receita_primaria.csv', encoding='utf8', decimal=",")

In [36]:
df.columns

Index(['Unnamed: 0', 'Ano', 'Esferas', 'Despesas em %'], dtype='object')

In [54]:
df = pd.read_csv('IPEA_despesa_PIB_salarios_RPPS.csv', encoding='utf8', decimal=",")

In [55]:
df=df.drop(['Unnamed: 0'],axis=1)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ano              38 non-null     int64  
 1   Tipo de despesa  38 non-null     object 
 2   % PIB            38 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ KB


In [57]:
df=df.dropna()

In [40]:
df.dtypes

Ano                  int64
Tipo de despesa     object
% PIB              float64
dtype: object

In [60]:
df.rename(columns={"Tipo de despesa": "Tipo_despesa", "% PIB":"percent_PIB"},inplace=True)

In [61]:
df.columns

Index(['Ano', 'Tipo_despesa', 'percent_PIB'], dtype='object')

#### Exemplo de job para subir uma tabela no GBQ

In [66]:
# set up BigQuery client and dataset reference

# Referencia do dataset
dataset_ref = client.dataset('conjunto_teste2')

In [68]:
## Cria o esquema da tabela

schema = [
    bigquery.SchemaField("Ano", "INTEGER"),
    bigquery.SchemaField("Tipo_despesa", "STRING"),
    bigquery.SchemaField("percent_PIB", "FLOAT")
]

In [69]:
# nome da tabela 
table_ref = dataset_ref.table('teste_2604')

In [72]:
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

NameError: name 'df2' is not defined

In [ ]:
## Deletar tabela

client.delete_table('acoes_afirmativas.FLACSO_acoes_afirmativas_grafico2', not_found_ok=True)

## Criando datasets fixos

In [37]:
# Quais serão os datasets? 
# Por fonte? Por categoria? 

## Nomes possíveis 

nomes_datasets= ["perfil_remuneracao", "acoes_afirmativas", "atracao_pre_selecao", "cargos_lideranca", "despesas_estado", "participacao_transparencia", "estrutura_organizacional_carreiras","legislacoes",
"percepcao_mensuracao_qualidade_estado","performance_valorizacao_engajamento"]

In [38]:
len(nomes_datasets)


10

In [12]:
client = bigquery.Client()

'repositoriodedadosgpsp'

In [24]:
dataset_id = "repositoriodedadosgpsp.{}".format(i)

In [15]:
for i in nomes_datasets:
    dataset_id = "repositoriodedadosgpsp.{}".format(i)
    # Transforma em google.cloud.bigquery.dataset.Dataset 
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    dataset = client.create_dataset(dataset)  
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset repositoriodedadosgpsp.perfil_remuneracao
Created dataset repositoriodedadosgpsp.acoes_afirmativas
Created dataset repositoriodedadosgpsp.atracao_pre_selecao
Created dataset repositoriodedadosgpsp.cargos_lideranca
Created dataset repositoriodedadosgpsp.despesas_estado
Created dataset repositoriodedadosgpsp.participacao_transparencia
Created dataset repositoriodedadosgpsp.estrutura_organizacional_carreiras
Created dataset repositoriodedadosgpsp.legislacoes
Created dataset repositoriodedadosgpsp.percepcao_mensuracao_qualidade_estado
Created dataset repositoriodedadosgpsp.performance_valorizacao_engajamento


In [27]:
dataset_id = "repositoriodedadosgpsp.{}".format(teste)
    # Transforma em google.cloud.bigquery.dataset.Dataset 
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset)  
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset repositoriodedadosgpsp.Ana


## Instruções gerais padrão de nomeação das tabelas: 


Padrão do nome do arquivo para baixar: 

`[Fonte]_descricao_maximo_5palavras.csv`

Padrão do nome do arquivo para gráficos:

`[Fonte]_descricao_maximo_5palavras_grafico.csv`




